# SEGUNDO ENFOQUE DE MI PROYECTO
Sigo con problema de Regresión, pero acoto los vinos manteniendo solo aquellos que cuestan hasta máximo 50$

            (basándome en mi story telling... 
            Ahora con la pandemia la gente más que quedar en bares queda en las casas y lleva la clásica botella de vino. 
            Quiero crear un recomendador que, introduciendo la puntuación de un vino, el año de producción y la variedad, 
            me prediga el precio, para yo saber sobre qué rango moverme si voy a casa de un amigo y le quiero regalar algun vino bueno. 
            Y por supuesto, por mucho que quiera a este amigo, sé que nunca me gastaría más de 50$ para ello. 
            O mismamente si la quiero usar para recomendar a gente ciertos vinos y decirle sobre qué precio se podría mover la cosa, 
            sé que en general un tope de 50$ sería más que suficiente por el entorno en el que me muevo)

            - Anteriormente he visto que me daba un mejor score(pero ligeramente mejor) usando la columna "points en vez que "ratings"
            Pero es verdad que normalmente en una aplicación si tienes que escoger parámetros, es mucho más fácils escoger entre un ratio de 1 a 5
            que de 80 a 100 (como estaban originariamente los puntos), por lo tanto escojo este dataframe, él que guardé con la nueva columna
            "ratings que creé a partir de la de "points"

In [1]:

## Primero, cargo todas las librerias 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

import re

from scipy.stats import iqr   ### para función que quita los outliers

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedKFold, ParameterGrid
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.svm import SVR
import xgboost


## PARA LAS TRANSFORMACIONES
from scipy.stats import shapiro
from scipy.stats import skew
from scipy import stats, special



import warnings
warnings.filterwarnings('once')

In [5]:
## cargo el dataframe ya limpio del primer análisis, el que ya tenia los valores pasados a numéricos y sin outliers
df = pd.read_csv("../data/processed/df_no_outliers.csv")

In [6]:
df.head()

,country_num,variety_num,year,price,ratings
0,0,7,2013,13.0,2
1,0,1,2012,65.0,2
2,3,208,2011,15.0,2
3,2,143,2013,16.0,2
4,1,18,2012,27.0,2


In [7]:
df.info()  ## 46982 elementos de partida, a ver con cuántos me quedaré ahora

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46982 entries, 0 to 46981
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country_num  46982 non-null  int64  
 1   variety_num  46982 non-null  int64  
 2   year         46982 non-null  int64  
 3   price        46982 non-null  float64
 4   ratings      46982 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 1.8 MB


In [9]:
## no voy a repetir el gráfico, lo tengo en el primer notebook junto con los demás estudios
## me limito a ordenarlo por precio descendiente y, en la siguiente celda, borraré los que cuesten más de 50$
df["price"].sort_values(ascending=False)

33106    250.0
41678    250.0
44539    250.0
43813    250.0
42721    250.0
         ...  
9019       5.0
791        4.0
41086      4.0
11282      4.0
42602      4.0
Name: price, Length: 46982, dtype: float64

In [12]:
df_50USD = df[df["price"] <= 50.0].sort_values(by="price", ascending=False)
df_50USD

,country_num,variety_num,year,price,ratings
16739,3,10,2007,50.0,3
19127,0,1,2010,50.0,2
19405,1,1,2014,50.0,4
40190,0,1,2012,50.0,4
19398,0,1,2014,50.0,4
...,...,...,...,...,...
23008,3,10,2012,5.0,1
42602,3,14,2012,4.0,1
791,3,6,2013,4.0,2
11282,4,185,2009,4.0,2


In [14]:
## como en el anterior notebook ví que en este problema de regresión era mejor si quitaba la columna del País, ya le hago un drop y directamente guardo este .csv, para 
## que sea más cómodo de manejar a la hora de probar los modelos de ML

df_50USD.drop(["country_num"], axis = 1, inplace= True)

In [15]:
df_50USD 
### me he quedado con 37176 registros, voy a guardar el csv

,variety_num,year,price,ratings
16739,10,2007,50.0,3
19127,1,2010,50.0,2
19405,1,2014,50.0,4
40190,1,2012,50.0,4
19398,1,2014,50.0,4
...,...,...,...,...
23008,10,2012,5.0,1
42602,14,2012,4.0,1
791,6,2013,4.0,2
11282,185,2009,4.0,2


In [17]:
df_50USD.to_csv("..\\data\\processed\\vinos_hasta_50USD.csv", index=False) 

# MODELOS DE ML

### 1º MODELO - GRADIENTBOOSTING 

- Score del modelo: 0.3886
- MAE: 7.2239
- MSE: 81.105
- RMSE  9.0058
- MAPE score: 27.4605

In [20]:
## defino "X" e "y"
X1 = df_50USD[["ratings", "year", "variety_num"]] 
y1 = np.array(df_50USD["price"]).reshape(-1,)

## separo en train y test
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2, random_state=12)


## escalo los datos
scal = StandardScaler() # Declaro el scaler
scal.fit(X1_train) # Lo entreno
X1_train = scal.transform(X1_train) # Aplico el scaler y sobreescribo los datos de train

# Aplico el mismo scaler con los datos de test
X1_test = scal.transform(X1_test)


grboost1 = GradientBoostingRegressor(max_depth=5,
                                 n_estimators=200,
                                 learning_rate=0.1,
                                 random_state=42)

grboost1.fit(X1_train, y1_train)


predictions1 = grboost1.predict(X1_test)
predictions1


# funcion para calcular el MAPE
def mean_absolute_percentage_error(y_pred, y_true): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape1 = round(mean_absolute_percentage_error(y1_test, predictions1), 4)


print('Score del modelo:', round(grboost1.score(X1_test, y1_test),4))
print('MAE:', round(mean_absolute_error(y1_test, predictions1),4))
print('MSE:', round(mean_squared_error(y1_test, predictions1),4))
print('RMSE ', round(np.sqrt(mean_squared_error(y1_test, predictions1)),4))
print("MAPE score:", mape1)


Score del modelo: 0.3886
MAE: 7.2239
MSE: 81.105
RMSE  9.0058
MAPE score: 27.4605


In [ ]:
## NOTAS:
## Al acotar tanto los precios el RMSE ha mejorado considerablemente (aun siendo 9, a mi parecer, un valor alto de error, pero seguramente mejor que 21)
## Por otro lado el score ha bajado a un 38%, cuando antes, aun teniendo peor RSME , tenía un 44% de score

# Hago otra par de prueba con otro modelo y ya me pasaré al tercer enfoque de mi proyecto, lo de pasarlo a clasificación

### 2º MODELO - XGBOOST 

- Score del modelo: 0.3887
- MAE: 7.2257
- MSE: 81.0935
- RMSE  9.0052
- MAPE score: 27.479

In [22]:

## defino "X" e "y"
X2 = df_50USD[["ratings", "year", "variety_num"]] 
y2 = np.array(df_50USD["price"]).reshape(-1,)

## separo en train y test
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state=12)


## escalo los datos
scal = StandardScaler() # Declaro el scaler
scal.fit(X2_train) # Lo entreno
X2_train = scal.transform(X2_train) # Aplico el scaler y sobreescribo los datos de train

# Aplico el mismo scaler con los datos de test
X2_test = scal.transform(X2_test)


xgb2 = xgboost.XGBRegressor(max_depth=5,
                                 n_estimators=200,
                                 learning_rate=0.1,
                                random_state=42)

xgb2.fit(X2_train, y2_train)

predictions2 = xgb2.predict(X2_test)
predictions2


# funcion para calcular el MAPE
def mean_absolute_percentage_error(y_pred, y_true): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape2 = round(mean_absolute_percentage_error(y2_test, predictions2), 4)


print('Score del modelo:', round(xgb2.score(X2_test, y2_test),4))
print('MAE:', round(mean_absolute_error(y2_test, predictions2),4))
print('MSE:', round(mean_squared_error(y2_test, predictions2),4))
print('RMSE ', round(np.sqrt(mean_squared_error(y2_test, predictions2)),4))
print("MAPE score:", mape2 )

Score del modelo: 0.3887
MAE: 7.2257
MSE: 81.0935
RMSE  9.0052
MAPE score: 27.479


In [ ]:
## NOTAS:
## Pasa algo muy parecido al anterior modelo, así que no tiene sentido que siga perdiendo el tiempo con más pruebas
## Al tener esta diferencia entre RMSE y score, me parece que los resultados no son concluyentes para lo que es el objetivo de mi proyecto
## ASÍ QUE ME PASO AL SIGUIENTE ENFOQUE, O SEA A UN PROBLEMA DE CLASIFICACIÓN (todos los códigos en el siguiente notebook)